# 4colab

In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# result_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/result'

In [2]:
data_dir = 'data'
result_dir = 'result'

# Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [4]:
af = 'vv'
pf = '__1__'

# Xtrain = np.load(f'{data_dir}/{af}3.tr.X.{pf}.npy', allow_pickle=True)
# y_train = np.load(f'{data_dir}/{af}3.tr.y.npy', allow_pickle=True)

Xtest = np.load(f'{data_dir}/{af}3.t.X.{pf}.npy', allow_pickle=True)
y_test = np.load(f'{data_dir}/{af}3.t.y.npy', allow_pickle=True)

# Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
# y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [5]:
# Xtrain.shape

In [6]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Test and evaluate

In [7]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    #? relabel to use classification metrics
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1

    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [8]:
import pickle

def save_result(model, Xt, yt_pred, scores, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    # if ra <= 70: #? not good enough to even bother
    #     return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.model.pkl','wb'))

    #? save prediction
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.result.t.y_pred.npy', yt_pred)
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.result.t.scores.npy', scores)

    #? save transformed X as well
    # np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.tr.X.npy', Xtr)
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.t.X.npy', Xt)
    # np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.v.X.npy', Xv)

    #? save fts
    open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.fts.txt', 'w').write('\n'.join(selected_fts_names))
    
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.config.txt', 'w').write(config)

## Exp 01: Run on original features (0-8)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`

Using only numeric features from original data, the result is not good

```
CPU times: total: 12min 24s
Wall time: 1min 21s

[[754066   7514]
 [  2830    132]]
              precision    recall  f1-score   support

           0     0.9963    0.9901    0.9932    761580
           1     0.0173    0.0446    0.0249      2962

    accuracy                         0.9865    764542
   macro avg     0.5068    0.5173    0.5090    764542
weighted avg     0.9925    0.9865    0.9894    764542

0.5173490764668689
```

In [9]:
""" select features """
selected_fts_names__01 = fts_names[0:8]
', '.join(selected_fts_names__01)

'Dur, sTos, dTos, Sport, Dport, PktsPerSec, BytesPerSec, SrcBytesPerSec'

In [19]:
# X_train__01 = Xtrain[:,0:8]
X_test__01 = Xtest[:,0:8]
# X_val__01 = Xval[:,0:8]

print(X_test__01.shape)

(764542, 8)


In [32]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__01 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [33]:
%%time
""" predict """
y_pred__01 = model__01.fit_predict(X_test__01)

CPU times: total: 12min 24s
Wall time: 1min 21s


In [34]:
""" evaluate """
roc_auc__01 = evaluate(y_test, y_pred__01)

[[754066   7514]
 [  2830    132]]
              precision    recall  f1-score   support

           0     0.9963    0.9901    0.9932    761580
           1     0.0173    0.0446    0.0249      2962

    accuracy                         0.9865    764542
   macro avg     0.5068    0.5173    0.5090    764542
weighted avg     0.9925    0.9865    0.9894    764542

0.5173490764668689


In [35]:
""" get lof scores """
scores_pred__01 = model__01.negative_outlier_factor_

In [36]:
""" save model """
save_result(model__01, X_test__01, y_pred__01, scores_pred__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [37]:
""" clean """
del X_test__01
del model__01, y_pred__01, scores_pred__01

## Exp 02: Run on numeric features (0-14)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `TotPkts`, `TotBytes`, `SrcBytes`, `DstBytes`, `DstBytesPerSec`

Using only numeric features, the result is not good.  
The result does not change much from Exp01

```
CPU times: total: 23min 28s
Wall time: 2min 33s

[[754045   7535]
 [  2851    111]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9932    761580
           1     0.0145    0.0375    0.0209      2962

    accuracy                         0.9864    764542
   macro avg     0.5054    0.5138    0.5070    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.5137903872469258
```

In [38]:
""" select features """
selected_fts_names__02 = fts_names[0:14]
', '.join(selected_fts_names__02)

'Dur, sTos, dTos, Sport, Dport, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, TotPkts, TotBytes, SrcBytes, DstBytes, DstBytesPerSec'

In [39]:
# X_train__02 = Xtrain[:,0:14]
X_test__02 = Xtest[:,0:14]
# X_val__02 = Xval[:,0:14]

print(X_test__02.shape)

(764542, 14)


In [40]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__02 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [41]:
%%time
""" predict """
y_pred__02 = model__02.fit_predict(X_test__02)

CPU times: total: 23min 28s
Wall time: 2min 33s


In [42]:
""" evaluate """
roc_auc__02 = evaluate(y_test, y_pred__02)

[[754045   7535]
 [  2851    111]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9932    761580
           1     0.0145    0.0375    0.0209      2962

    accuracy                         0.9864    764542
   macro avg     0.5054    0.5138    0.5070    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.5137903872469258


In [43]:
""" get lof scores """
scores_pred__02 = model__02.negative_outlier_factor_

In [44]:
""" save model """
save_result(model__02, X_test__02, y_pred__02, scores_pred__02, 'lof', selected_fts_names__02, roc_auc__02, 'exp02_fts-0-14', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [48]:
""" clean """
del X_test__02
del model__02, y_pred__02, scores_pred__02

## Exp 03: Run on all features

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `TotPkts`, `TotBytes`, `SrcBytes`, `DstBytes`, `DstBytesPerSec`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result actually decrease ?!

```
CPU times: total: 1h 52min 32s
Wall time: 8min 14s

[[754007   7573]
 [  2889     73]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0095    0.0246    0.0138      2962

    accuracy                         0.9863    764542
   macro avg     0.5029    0.5074    0.5034    764542
weighted avg     0.9924    0.9863    0.9893    764542

0.5073508543727432
```

In [9]:
"', '".join(fts_names)

"Dur', 'sTos', 'dTos', 'Sport', 'Dport', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'TotPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'DstBytesPerSec', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other', 'sTos_0', 'sTos_2', 'sTos_3', 'sTos_1', 'sTos_-1', 'dTos_0', 'dTos_-1', 'dTos_3', 'dTos_2', 'dTos_1"

In [10]:
""" select features """
selected_fts_names__03 = ['Dur', 'sTos', 'dTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'DstBytes', 'DstBytesPerSec', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__03 = [fts_names.index(s) for s in selected_fts_names__03]
len(idx__selected_fts_names__03)

53

In [51]:
# X_train__03 = Xtrain[:,idx__selected_fts_names__03]
X_test__03 = Xtest[:,idx__selected_fts_names__03]
# X_val__03 = Xval[:,idx__selected_fts_names__03]

print(X_test__03.shape)

(764542, 53)


In [52]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__03 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [53]:
%%time
""" predict """
y_pred__03 = model__03.fit_predict(X_test__03)

CPU times: total: 1h 52min 32s
Wall time: 8min 14s


In [57]:
""" evaluate """
roc_auc__03 = evaluate(y_test, y_pred__03)

[[754007   7573]
 [  2889     73]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0095    0.0246    0.0138      2962

    accuracy                         0.9863    764542
   macro avg     0.5029    0.5074    0.5034    764542
weighted avg     0.9924    0.9863    0.9893    764542

0.5073508543727432


In [54]:
""" get lof scores """
scores_pred__03 = model__03.negative_outlier_factor_

In [58]:
""" save model """
save_result(model__03, X_test__03, y_pred__03, scores_pred__03, 'lof', selected_fts_names__03, roc_auc__03, 'exp03_fts-all', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [59]:
""" clean """
del X_test__03
del model__03, y_pred__03, scores_pred__03

## Exp 04: Run on chosen set of features

`sTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result is just similar as Exp01.  

```
CPU times: total: 1h 46min 15s
Wall time: 7min 31s

[[754045   7535]
 [  2851    111]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9932    761580
           1     0.0145    0.0375    0.0209      2962

    accuracy                         0.9864    764542
   macro avg     0.5054    0.5138    0.5070    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.5137903872469258
```

In [60]:
""" select features """
selected_fts_names__04 = ['sTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__04 = [fts_names.index(s) for s in selected_fts_names__04]
len(idx__selected_fts_names__04)

49

In [61]:
# X_train__04 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
X_test__04 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
# X_val__04 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)

print(X_test__04.shape)

(764542, 49)


In [62]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__04 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [63]:
%%time
""" predict """
y_pred__04 = model__04.fit_predict(X_test__04)

CPU times: total: 1h 46min 15s
Wall time: 7min 31s


In [64]:
""" evaluate """
roc_auc__04 = evaluate(y_test, y_pred__04)

[[754045   7535]
 [  2851    111]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9932    761580
           1     0.0145    0.0375    0.0209      2962

    accuracy                         0.9864    764542
   macro avg     0.5054    0.5138    0.5070    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.5137903872469258


In [65]:
""" get lof scores """
scores_pred__04 = model__04.negative_outlier_factor_

In [66]:
""" save model """
save_result(model__04, X_test__04, y_pred__04, scores_pred__04, 'lof', selected_fts_names__04, roc_auc__04, 'exp04_play', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [67]:
""" clean """
del X_test__04
del model__04, y_pred__04, scores_pred__04

## Exp 05: Run on numeric features chosen from MI + one-hot features

**Chosen features from MI: `dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `TotBytes`, `DstBytes`**

```
CPU times: total: 1h 49min 1s
Wall time: 7min 18s

[[754033   7547]
 [  2863     99]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0129    0.0334    0.0187      2962

    accuracy                         0.9864    764542
   macro avg     0.5046    0.5118    0.5059    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.5117568505498156
```

In [68]:
""" select features """
selected_fts_names__05 = ['dTos', 'Sport', 'BytesPerPkt', 'Dur', 'BytesPerSec', 'DstBytesPerSec', 'PktsPerSec', 'SrcBytesPerSec', 'TotBytes', 'DstBytes', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__05 = [fts_names.index(s) for s in selected_fts_names__05]
len(idx__selected_fts_names__05)

49

In [69]:
# X_train__05 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
X_test__05 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
# X_val__05 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)

print(X_test__05.shape)

(764542, 49)


In [70]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__05 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [71]:
%%time
""" predict """
y_pred__05 = model__05.fit_predict(X_test__05)

CPU times: total: 1h 49min 1s
Wall time: 7min 18s


In [72]:
""" evaluate """
roc_auc__05 = evaluate(y_test, y_pred__05)

[[754033   7547]
 [  2863     99]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0129    0.0334    0.0187      2962

    accuracy                         0.9864    764542
   macro avg     0.5046    0.5118    0.5059    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.5117568505498156


In [73]:
""" get lof scores """
scores_pred__05 = model__05.negative_outlier_factor_

In [74]:
""" save model """
save_result(model__05, X_test__05, y_pred__05, scores_pred__05, 'lof', selected_fts_names__05, roc_auc__05, 'exp05_mi1', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [75]:
""" clean """
del X_test__05
del model__05, y_pred__05, scores_pred__05

## Exp 06: Run on features chosen from MI

`State_S_` ,` dTos` ,` Proto_tcp` ,` Proto_udp` ,` Service_80` ,` Service_other` ,` State_CON` ,` Service_25` ,` Sport` ,` Flag_S` ,` Flag_R` ,` BytesPerPkt` ,` Dur` ,` Service_6667` ,` BytesPerSec` ,` State_alltcp` ,` DstBytesPerSec` ,` Flag_A` ,` PktsPerSec` ,` SrcBytesPerSec`

```
CPU times: total: 1h 20min 59s
Wall time: 5min 32s

[[754030   7550]
 [  2866     96]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0126    0.0324    0.0181      2962

    accuracy                         0.9864    764542
   macro avg     0.5044    0.5112    0.5056    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.511248466375538
```

In [76]:
""" select features """
# selected_fts_names__06 = fts_names[29:30] + fts_names[2:3] + fts_names[51:52] + fts_names[50:51] + fts_names[58:59] + fts_names[64:65] + fts_names[26:27] + fts_names[62:63] + fts_names[3:4] + fts_names[45:46] + fts_names[48:49] + fts_names[26:65]
selected_fts_names__06 = ['State_S_','dTos','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Sport','Flag_S','Flag_R','BytesPerPkt','Dur','Service_6667','BytesPerSec','State_alltcp','DstBytesPerSec','Flag_A','PktsPerSec','SrcBytesPerSec']
idx__selected_fts_names__06 = [fts_names.index(s) for s in selected_fts_names__06]
len(idx__selected_fts_names__06)
# ', '.join(selected_fts_names__05)

20

In [77]:
# X_train__06 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
X_test__06 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
# X_val__06 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)

print(X_test__06.shape)

(764542, 20)


In [78]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__06 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [79]:
%%time
""" predict """
y_pred__06 = model__06.fit_predict(X_test__06)

CPU times: total: 1h 20min 59s
Wall time: 5min 32s


In [80]:
""" evaluate """
roc_auc__06 = evaluate(y_test, y_pred__06)

[[754030   7550]
 [  2866     96]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0126    0.0324    0.0181      2962

    accuracy                         0.9864    764542
   macro avg     0.5044    0.5112    0.5056    764542
weighted avg     0.9924    0.9864    0.9894    764542

0.511248466375538


In [81]:
""" get lof scores """
scores_pred__06 = model__06.negative_outlier_factor_

In [84]:
""" save model """
save_result(model__06, X_test__06, y_pred__06, scores_pred__06, 'lof', selected_fts_names__06, roc_auc__06, 'exp06_mi2', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [85]:
""" clean """
del X_test__06
del model__06, y_pred__06, scores_pred__06

## Exp 07: Run on numeric features chosen from MI + ohe features

Chosen features from MI: `dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `State_S_`, `Proto_tcp`, `Proto_udp`, `Service_80`, `Service_other`, `State_CON`, `Service_25`, `Flag_S`

```
CPU times: total: 1h 9min 56s
Wall time: 5min 20s

[[754008   7572]
 [  2888     74]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0097    0.0250    0.0140      2962

    accuracy                         0.9863    764542
   macro avg     0.5029    0.5075    0.5035    764542
weighted avg     0.9924    0.9863    0.9893    764542

0.5075203157641691
```

In [11]:
""" select features """
selected_fts_names__07 = ['dTos','Sport','BytesPerPkt','Dur','BytesPerSec','DstBytesPerSec','PktsPerSec','SrcBytesPerSec',
    'State_S_','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Flag_S'
]
idx__selected_fts_names__07 = [fts_names.index(s) for s in selected_fts_names__07]
len(idx__selected_fts_names__07)

16

In [12]:
# X_train__07 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)
X_test__07 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)
# X_val__07 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)

print(X_test__07.shape)

(764542, 16)


In [13]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 65
contamination = 0.01
model__07 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, n_jobs=10)

In [14]:
%%time
""" predict """
y_pred__07 = model__07.fit_predict(X_test__07)

CPU times: total: 1h 9min 56s
Wall time: 5min 20s


In [15]:
""" evaluate """
roc_auc__07 = evaluate(y_test, y_pred__07)

[[754008   7572]
 [  2888     74]]
              precision    recall  f1-score   support

           0     0.9962    0.9901    0.9931    761580
           1     0.0097    0.0250    0.0140      2962

    accuracy                         0.9863    764542
   macro avg     0.5029    0.5075    0.5035    764542
weighted avg     0.9924    0.9863    0.9893    764542

0.5075203157641691


In [18]:
""" get lof scores """
scores_pred__07 = model__07.negative_outlier_factor_

In [19]:
""" save model """
save_result(model__07, X_test__07, y_pred__07, scores_pred__07, 'lof', selected_fts_names__07, roc_auc__07, 'exp07_mi3', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [20]:
""" clean """
del X_test__07
del model__07, y_pred__07, scores_pred__07